# FGSM (Fast Gradient Sign Method) Adversarial Attack
This notebook demonstrates how to run a basic FGSM adversarial attack using TensorFlow and the Adversarial Robustness Toolbox (ART).

⚠️ CPU-only, light, and beginner-friendly.

**Purpose:** Practice and understand adversarial attacks using the FGSM method.


In [ ]:
# Step 1: Import required libraries
# These should already be installed in your virtual environment
import numpy as np
import tensorflow as tf
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_cifar10


In [ ]:
# Step 2: Load dataset (CIFAR-10)
# You can replace this with your own dataset if preferred
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()
x_train, x_test = x_train[:500], x_test[:100]  # reduce size for speed
y_train, y_test = y_train[:500], y_test[:100]

In [ ]:
# Step 3: Define a simple CNN model (light and fast for CPU)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=1)

In [ ]:
# Step 4: Wrap the model with ART’s TensorFlowV2Classifier
classifier = TensorFlowV2Classifier(
    model=model,
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    train_step=None,
    nb_classes=10,
    input_shape=(32, 32, 3),
    clip_values=(min_pixel_value, max_pixel_value)
)

In [ ]:
# Step 5: Create and apply the FGSM attack
attack = FastGradientMethod(estimator=classifier, eps=0.1)
x_test_adv = attack.generate(x=x_test)

In [ ]:
# Step 6: Evaluate the model on adversarial examples
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
true = np.argmax(y_test, axis=1)
acc = np.sum(preds == true) / len(true)
print(f"Accuracy on adversarial test examples: {acc * 100:.2f}%")